# The basics of querying CMR
The simplest query to CMR is a collection-level search with no filter constraints:

    GET https://cmr.earthdata.nasa.gov/search/collections

We will use this query to demonstrate the basic functionality of CMR search.
What can we do with this query?

In [9]:
import requests
import xml.dom.minidom

# The simplest search request possible
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections")

The default response is an xml document of collection result references. Let's pretty print the output.

In [10]:
# The default response is an xml document of result references. Let's pretty print the output
response_as_dom = xml.dom.minidom.parseString(response.text)
xml_reference_response = response_as_dom.toprettyxml()
print(xml_reference_response)

<?xml version="1.0" ?>
<results>
	<hits>28348</hits>
	<took>9</took>
	<references>
		<reference>
			<name>&quot;The Omnivores Dilemma&quot;: The Effect of Autumn Diet on Winter Physiology and Condition of Juvenile Antarctic Krill</name>
			<id>C1934541400-SCIOPS</id>
			<location>https://cmr.earthdata.nasa.gov:443/search/concepts/C1934541400-SCIOPS/1</location>
			<revision-id>1</revision-id>
		</reference>
		<reference>
			<name>'Latent reserves' within the Swiss NFI</name>
			<id>C1931110427-SCIOPS</id>
			<location>https://cmr.earthdata.nasa.gov:443/search/concepts/C1931110427-SCIOPS/5</location>
			<revision-id>5</revision-id>
		</reference>
		<reference>
			<name>(U-Th)/He ages from the Kukri Hills of southern Victoria Land</name>
			<id>C1214587974-SCIOPS</id>
			<location>https://cmr.earthdata.nasa.gov:443/search/concepts/C1214587974-SCIOPS/4</location>
			<revision-id>4</revision-id>
		</reference>
		<reference>
			<name>0.5 hour 1 M HCl extraction data for the Windmill Islands

## Parsing the xml results
We can extract information from the results in a programatic way using xpath

How many collection results are there?

In [22]:
import xml.etree.ElementTree as et
doc = et.fromstring(response.text)
# Hits
print("Total number of collection results: " + doc.findtext('hits'))   


Total number of collection results: 28348


How many collection results were returned?

In [23]:
# No. of results returned
references = doc.findall('references/reference')
print("No. of results returned: " + str(len(references)))  


No. of results returned: 20


Where is the first collection result?

In [24]:
# First reference
print("First result reference: " + str(references[0].find('location').text))

First result reference: https://cmr.earthdata.nasa.gov:443/search/concepts/C1214610401-SCIOPS/3


What is the human-readable name of the first collection result?

In [25]:
# First name
print("First result name: " + str(references[0].find('name').text))

First result name: 1982 Commodity Output by State and Input-Output Sector


What is the unique ID (concept-id) of the first collection result?

In [26]:
print("First result concept id: " + str(references[0].find('id').text))

First result concept id: C1214610401-SCIOPS


## Going further than the first 10 results
CMR search supports paging parameters to iterate through 'pages' of results.
You can select your page size and your page number. Page size can be a positive integer between 1 and 2000. Page number can be a positive integer.
Note: for harvesting use cases we do not recommend paging through results. See: foo
Let's try getting 20 results instead of the default value of 10.

In [27]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?page_size=20")
doc = et.fromstring(response.text)
print("No. of results returned: " + str(len(references))) 
references = doc.findall('references/reference')
print("11th result concept id: " + str(references[10].find('id').text)) 

No. of results returned: 20
11th result concept id: C1214422215-SCIOPS


We have retrieved the first 20 results instead of the default 10.
Now let's try getting the next 20 results

In [28]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?page_size=20&page_num=2")
doc = et.fromstring(response.text)
references = doc.findall('references/reference')
print("11th result concept id: " + str(references[10].find('id').text)) 

11th result concept id: C1214610584-SCIOPS


Notice that the 11th result is different because we are looking at a different page

In [29]:
## Getting metadata by reference

## Getting metadata by search constraints

## Getting metadata in the format you want